# Classify the ICESAT-2 Photons

The next stage is to classify the ICESAT-2 photons to identify the bathymetry points from the ATL03. To achieve these we'll use the C-SHELPh module (https://github.com/nmt28/C-SHELPh).


## Import Modules

In [ ]:
import pathlib
import cshelph.run_cshelph
import json

## Iterate through Data Directories

In [ ]:
base_data_dir = "icesat2_dwnld"

In [ ]:
base_path = pathlib.Path(base_data_dir)

In [ ]:
roi_paths = [f for f in base_path.iterdir() if f.is_dir()]

In [ ]:
for roi_path in roi_paths:
    year_paths = [f for f in roi_path.iterdir() if f.is_dir()]
    out_info = dict()
    for year_path in year_paths:
        h5_paths = year_path.glob("*.h5")
        for h5_path in h5_paths:
            print(h5_path)
            h5_filename = h5_path.stem
            print(f"\t{h5_filename}")
            out_info[h5_filename] = dict()
            for laser in [1, 2, 3]:
                try:
                    cshelph.run_cshelph.run_cshelph(
                        input_h5_file = str(h5_path),
                        laser_num = laser,
                        thresh = 20,
                        threshlist = None,
                        output = None,
                        lat_res = 10,
                        h_res = 0.5,
                        water_temp = 25,
                        start_lat = None,
                        end_lat = None,
                        min_buffer = -40,
                        max_buffer = 5,
                        surface_buffer = -0.5,
                    )
                    out_info[h5_filename][laser] = "OK"
                except Exception as e:
                    out_info[h5_filename][laser] = str(e)
    out_info_file = roi_path / "cshelph_processing.json"
    with open(out_info_file, "w") as fp:
        json.dump(
            out_info,
            fp,
            sort_keys=True,
            indent=4,
            separators=(",", ": "),
            ensure_ascii=False,
        )